In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

path = '/content/drive/MyDrive/Obeject_trajectory_research'
print(os.listdir(path))

['Part I', 'Part II']


In [ ]:
import pandas as pd
df_data_points = pd.read_excel('/content/drive/MyDrive/Obeject_trajectory_research/Random_coordinates.xlsx')
df_data_points.head()

,Iteration,Time Point (t),X Coordinate,Y Coordinate
0,1,3.362509,46.924917,87.457296
1,1,9.237389,84.114013,94.411975
2,1,9.725993,90.464774,60.825287
3,1,9.956909,3.755938,59.665541
4,1,10.286336,50.831545,78.364425


In [ ]:
df_distances = pd.read_excel('/content/drive/MyDrive/Obeject_trajectory_research/Distances.xlsx')[['Distances']]
df_distances.head()

,Distances
0,1291.451524
1,1236.652469
2,884.001201
3,381.396786
4,1122.314349


In [ ]:
df_points_list = pd.read_excel('/content/drive/MyDrive/Obeject_trajectory_research/points_list.xlsx')
df_points_list.head()

,u,v,w
0,0.0,0.300000,0.0
1,0.0,0.250000,0.0
2,0.0,0.208333,0.0
3,0.0,0.173611,0.0
4,0.0,0.144676,0.0


In [ ]:
trajectory = df_data_points[df_data_points['Iteration'] == 1]
x = trajectory['X Coordinate']
y = trajectory['Y Coordinate']
t = trajectory['Time Point (t)']

+ Gaussian Process Regressor & Fourier Transform

In [ ]:
!pip install gpytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.8/277.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 18.4 MB/s eta 0:00:00


In [ ]:
import torch
import gpytorch
import numpy as np

In [ ]:
# Define the exact GP model with custom kernel
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)

        # Mean module (constant mean)
        self.mean_module = gpytorch.means.ConstantMean()

        # Covariance (kernel) module using C(1.0, (1e-3, 1e3)) * RBF(1.0, (0.5, 0.5))
        # Constant Kernel (C) multiplied by RBF (Radial Basis Function) Kernel
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(lengthscale=1.0, lengthscale_constraint=gpytorch.constraints.Interval(0.4, 0.6))
        )

        # Set the initial value of the outputscale (constant scaling factor C) and bounds
        self.covar_module.outputscale = 1.0
        self.covar_module.outputscale_constraint = gpytorch.constraints.Interval(1e-3, 1e3)

    def forward(self, x):
        mean_x = self.mean_module(x)  # Calculate the mean of the GP
        covar_x = self.covar_module(x)  # Calculate the covariance (kernel) of the GP
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [ ]:
def train_exact_gp_model(train_x, train_y, num_iterations=50, lr=0.1):
    # Set up the likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)

    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_x, train_y, model, likelihood = train_x.to(device), train_y.to(device), model.to(device), likelihood.to(device)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Define the optimizer
    optimizer = torch.optim.Adam([{'params': model.parameters()}], lr=lr)

    # Define the "marginal log likelihood" loss
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Training loop
    for i in range(num_iterations):
        optimizer.zero_grad()  # Zero the gradients
        output = model(train_x)  # Forward pass through the model
        loss = -mll(output, train_y)  # Compute the loss
        loss.backward()  # Backpropagate the gradients
        optimizer.step()  # Update the model parameters

        #if i % 10 == 0:
        #    print(f'Iteration {i + 1}/{num_iterations} - Loss: {loss.item():.3f}')

    # Return the trained model
    return model, likelihood

In [ ]:
#test
train_x = torch.tensor(x, dtype=torch.float32)
train_t = torch.tensor(t, dtype=torch.float32)

trained_model_x, trained_likelihood_x = train_exact_gp_model(train_x, train_t, num_iterations=50, lr=0.1)

In [ ]:
#test
train_y = torch.tensor(y, dtype=torch.float32)
train_t = torch.tensor(t, dtype=torch.float32)

trained_model_y, trained_likelihood_y = train_exact_gp_model(train_y, train_t, num_iterations=50, lr=0.1)

In [ ]:
def integrand(t, u_batch, v_batch, w_batch, gpr_fit_x, gpr_fit_y, likelihood_x, likelihood_y):
    # Ensure `t` is a tensor and moved to the same device as u_batch
    t_tensor = torch.tensor([t], dtype=torch.float32).to(u_batch.device).expand_as(u_batch)

    # Put the model in evaluation mode
    gpr_fit_x.eval()
    gpr_fit_y.eval()
    likelihood_x.eval()
    likelihood_y.eval()

    with torch.no_grad():  # No gradients needed for prediction
        # Get predictive distribution for gpr_fit_x and gpr_fit_y for all batches
        pred_x = likelihood_x(gpr_fit_x(t_tensor))
        pred_y = likelihood_y(gpr_fit_y(t_tensor))

        # Extract mean and standard deviation
        mu_x = pred_x.mean
        sigma_x = pred_x.stddev

        mu_y = pred_y.mean
        sigma_y = pred_y.stddev

    # Compute real and imaginary parts for the entire batch
    real = torch.cos(2 * torch.pi * (mu_x * u_batch + mu_y * v_batch + w_batch * t_tensor)) * \
           torch.exp(-2 * torch.pi**2 * (sigma_x**2 * u_batch**2 + sigma_y**2 * v_batch**2))

    imag = -torch.sin(2 * torch.pi * (mu_x * u_batch + mu_y * v_batch + w_batch * t_tensor)) * \
           torch.exp(-2 * torch.pi**2 * (sigma_x**2 * u_batch**2 + sigma_y**2 * v_batch**2))

    return real, imag


In [ ]:
def fourier_transform_batch(u_vals, v_vals, w_vals, tmin, tmax, gpr_fit_x, gpr_fit_y, likelihood_x, likelihood_y, num_points=1000):
    # Generate equally spaced time points between tmin and tmax (on GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    t_vals = torch.linspace(tmin, tmax, num_points).to(device)

    # Convert u_vals, v_vals, w_vals to tensors and move them to GPU
    u_tensor = torch.tensor(u_vals, dtype=torch.float32).to(device)
    v_tensor = torch.tensor(v_vals, dtype=torch.float32).to(device)
    w_tensor = torch.tensor(w_vals, dtype=torch.float32).to(device)

    # Initialize real and imaginary parts (batch)
    real_vals = torch.zeros(num_points, len(u_vals), device=device)
    imag_vals = torch.zeros(num_points, len(u_vals), device=device)

    # Compute real and imaginary parts for all batches at each time point
    for i, t in enumerate(t_vals):
        real_batch, imag_batch = integrand(t, u_tensor, v_tensor, w_tensor, gpr_fit_x, gpr_fit_y, likelihood_x, likelihood_y)
        real_vals[i] = real_batch
        imag_vals[i] = imag_batch

    # Use the trapezoidal rule to integrate the real and imaginary parts
    real_part = torch.trapz(real_vals, t_vals, dim=0)
    imag_part = torch.trapz(imag_vals, t_vals, dim=0)

    return real_part, imag_part


In [ ]:
u_vals = df_points_list['u'].values
v_vals = df_points_list['v'].values
w_vals = df_points_list['w'].values

#real_part_batch, imag_part_batch = fourier_transform_batch(u_vals, v_vals, w_vals, min(t), max(t), trained_model_x, trained_model_y, trained_likelihood_x, trained_likelihood_y, num_points=1000)


Fourier Transform running

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results_arr_real = []
results_arr_imag = []
for ind in range(5501,6501):
    trajectory = df_data_points[df_data_points['Iteration'] == ind]
    x = trajectory['X Coordinate']
    y = trajectory['Y Coordinate']
    t = trajectory['Time Point (t)']

    train_x = torch.tensor(x.values, dtype=torch.float32).to(device)
    train_y = torch.tensor(y.values, dtype=torch.float32).to(device)
    train_t = torch.tensor(t.values, dtype=torch.float32).to(device)

    print(f"train_x device: {train_x.device}")

    trained_model_x, trained_likelihood_x = train_exact_gp_model(train_x, train_t, num_iterations=50, lr=0.1)
    trained_model_y, trained_likelihood_y = train_exact_gp_model(train_y, train_t, num_iterations=50, lr=0.1)

    print(f"trained_model_x device: {next(trained_model_x.parameters()).device}")
    print(f"trained_model_y device: {next(trained_model_y.parameters()).device}")

    real_part_batch, imag_part_batch = fourier_transform_batch(u_vals, v_vals, w_vals, min(t), max(t), trained_model_x, trained_model_y, trained_likelihood_x, trained_likelihood_y, num_points=1000)


    print(f"real_part device: {real_part_batch.device}, imag_part device: {imag_part_batch.device}")
    print(ind)

    results_arr_real.append(real_part_batch)
    results_arr_imag.append(imag_part_batch)


train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5501
train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5502
train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5503
train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5504
train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5505
train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5506
train_x device: cuda:0
trained_model_x device: cuda:0
trained_model_y device: cuda:0
real_part device: cuda:0, imag_part device: cuda:0
5507
train_x devic

In [ ]:
df_results_arr_real = pd.DataFrame(results_arr_real)
df_results_arr_real = df_results_arr_real.applymap(lambda x: x.item())
df_results_arr_real.head()

<ipython-input-15-417d442fa38d>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_results_arr_real = df_results_arr_real.applymap(lambda x: x.item())


,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,0.000293,0.008484,0.050428,-0.449594,-2.962220,-7.945268,-12.594744,-12.773037,-7.177463,2.092081,...,0.008484,0.050428,-0.449594,-2.962220,-7.945268,-12.594744,-12.773037,-7.177463,2.092081,89.688271
1,0.000123,0.006644,0.071249,-0.263965,-2.452215,-6.919966,-10.786085,-13.010109,-7.533419,0.375121,...,0.006644,0.071249,-0.263965,-2.452215,-6.919966,-10.786085,-13.010109,-7.533419,0.375121,82.279434
2,0.000252,0.006199,0.052172,-0.217462,-2.367059,-6.699406,-9.733435,-11.606384,-8.218695,0.590425,...,0.006199,0.052172,-0.217462,-2.367059,-6.699406,-9.733435,-11.606384,-8.218695,0.590425,89.752151
3,0.000175,0.012985,0.057817,-0.553840,-3.870724,-9.863118,-14.727495,-13.279825,-6.097941,6.054640,...,0.012985,0.057817,-0.553840,-3.870724,-9.863118,-14.727495,-13.279825,-6.097941,6.054640,87.896263
4,0.000144,0.010930,0.053647,-0.402822,-3.071263,-8.258067,-12.746071,-12.656931,-7.191512,3.577903,...,0.010930,0.053647,-0.402822,-3.071263,-8.258067,-12.746071,-12.656931,-7.191512,3.577903,82.304153


In [ ]:
len(df_results_arr_real)

1000

In [ ]:
df_results_arr_real.to_excel('/content/drive/MyDrive/Obeject_trajectory_research/df_results_arr_real_5500_6500.xlsx', index=False)

In [ ]:
df_results_arr_imag = pd.DataFrame(results_arr_imag)
df_results_arr_imag = df_results_arr_imag.applymap(lambda x: x.item())
df_results_arr_imag.head()

<ipython-input-18-9fbad7424e33>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_results_arr_imag = df_results_arr_imag.applymap(lambda x: x.item())


,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,-0.000355,0.000581,0.151732,0.829341,1.285801,-1.307926,-8.949072,-20.823704,-31.403984,-40.558083,...,-0.000581,-0.151732,-0.829341,-1.285801,1.307926,8.949072,20.823704,31.403984,40.558083,0.0
1,-0.000100,0.000418,0.098721,0.724610,1.483850,-0.444722,-6.810971,-16.401405,-28.575827,-34.742920,...,-0.000418,-0.098721,-0.724610,-1.483850,0.444722,6.810971,16.401405,28.575827,34.742920,0.0
2,0.000030,0.002623,0.129067,0.710916,1.063697,-0.443134,-8.176364,-15.314278,-24.572926,-36.694065,...,-0.002623,-0.129067,-0.710916,-1.063697,0.443134,8.176364,15.314278,24.572926,36.694065,0.0
3,-0.000285,0.003393,0.178802,1.060322,1.603191,-2.181261,-11.802144,-26.356705,-38.418243,-47.858082,...,-0.003393,-0.178802,-1.060322,-1.603191,2.181261,11.802144,26.356705,38.418243,47.858082,0.0
4,-0.000114,0.002522,0.136340,0.882885,1.297663,-0.839360,-9.082271,-21.117840,-32.507126,-42.875084,...,-0.002522,-0.136340,-0.882885,-1.297663,0.839360,9.082271,21.117840,32.507126,42.875084,0.0


In [ ]:
len(df_results_arr_imag)

1000

In [ ]:
df_results_arr_imag.to_excel('/content/drive/MyDrive/Obeject_trajectory_research/df_results_arr_imag_5500_6500.xlsx', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

path = '/content/drive/MyDrive/Obeject_trajectory_research'
print(os.listdir(path))

import pandas as pd
df_real = pd.read_excel('/content/drive/MyDrive/Obeject_trajectory_research/df_results_arr_real.xlsx')
df_combined_real = pd.concat([df_real, df_results_arr_real], axis=0, ignore_index=True)
df_combined_real

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Random_coordinates.xlsx', 'Distances.xlsx', 'points_list.xlsx', 'Multiple Random Object Trajectories (1).ipynb', 'MLP model.ipynb', 'Multiple Random Object Trajectories.ipynb', 'df_results_arr_imag.xlsx', 'df_results_arr_real.xlsx']


,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,0.000432,0.006607,0.039193,-0.143935,-1.767283,-6.526934,-11.184092,-12.711309,-9.456711,-1.641575,...,0.006607,0.039193,-0.143935,-1.767283,-6.526934,-11.184092,-12.711309,-9.456711,-1.641575,95.041718
1,-0.000030,0.011446,0.042073,-0.231437,-2.391988,-6.892663,-10.779109,-10.067949,-7.667257,-1.929023,...,0.011446,0.042073,-0.231437,-2.391988,-6.892663,-10.779109,-10.067949,-7.667257,-1.929023,96.968796
2,0.000256,0.008298,0.047472,-0.428711,-2.925494,-7.885454,-12.822762,-11.010488,-5.186280,2.668768,...,0.008298,0.047472,-0.428711,-2.925494,-7.885454,-12.822762,-11.010488,-5.186280,2.668768,98.098419
3,0.000104,0.010083,0.060578,-0.460531,-3.192271,-8.206146,-12.080729,-11.279043,-4.986012,4.267717,...,0.010083,0.060578,-0.460531,-3.192271,-8.206146,-12.080729,-11.279043,-4.986012,4.267717,73.169739
4,-0.000003,0.007162,0.045818,-0.339847,-2.584204,-6.577459,-10.365834,-10.993408,-7.023180,1.436533,...,0.007162,0.045818,-0.339847,-2.584204,-6.577459,-10.365834,-10.993408,-7.023180,1.436533,83.030594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0.000269,0.006116,0.054420,-0.273319,-2.128410,-5.646583,-8.764147,-9.577295,-6.524882,0.081393,...,0.006116,0.054420,-0.273319,-2.128410,-5.646583,-8.764147,-9.577295,-6.524882,0.081393,96.754868
3496,-0.000062,0.005809,0.021386,-0.166383,-1.704553,-4.992775,-7.245802,-7.599645,-7.360784,-3.224979,...,0.005809,0.021386,-0.166383,-1.704553,-4.992775,-7.245802,-7.599645,-7.360784,-3.224979,85.600113
3497,0.000243,0.011048,0.041259,-0.389725,-2.908340,-7.789240,-11.448001,-11.546152,-7.852053,0.080314,...,0.011048,0.041259,-0.389725,-2.908340,-7.789240,-11.448001,-11.546152,-7.852053,0.080314,95.613312
3498,0.000082,0.006800,0.033747,-0.100591,-1.874741,-5.169100,-8.544689,-10.632514,-6.163387,-4.961085,...,0.006800,0.033747,-0.100591,-1.874741,-5.169100,-8.544689,-10.632514,-6.163387,-4.961085,85.756920


In [ ]:
df_combined_real

,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,0.000432,0.006607,0.039193,-0.143935,-1.767283,-6.526934,-11.184092,-12.711309,-9.456711,-1.641575,...,0.006607,0.039193,-0.143935,-1.767283,-6.526934,-11.184092,-12.711309,-9.456711,-1.641575,95.041718
1,-0.000030,0.011446,0.042073,-0.231437,-2.391988,-6.892663,-10.779109,-10.067949,-7.667257,-1.929023,...,0.011446,0.042073,-0.231437,-2.391988,-6.892663,-10.779109,-10.067949,-7.667257,-1.929023,96.968796
2,0.000256,0.008298,0.047472,-0.428711,-2.925494,-7.885454,-12.822762,-11.010488,-5.186280,2.668768,...,0.008298,0.047472,-0.428711,-2.925494,-7.885454,-12.822762,-11.010488,-5.186280,2.668768,98.098419
3,0.000104,0.010083,0.060578,-0.460531,-3.192271,-8.206146,-12.080729,-11.279043,-4.986012,4.267717,...,0.010083,0.060578,-0.460531,-3.192271,-8.206146,-12.080729,-11.279043,-4.986012,4.267717,73.169739
4,-0.000003,0.007162,0.045818,-0.339847,-2.584204,-6.577459,-10.365834,-10.993408,-7.023180,1.436533,...,0.007162,0.045818,-0.339847,-2.584204,-6.577459,-10.365834,-10.993408,-7.023180,1.436533,83.030594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0.000269,0.006116,0.054420,-0.273319,-2.128410,-5.646583,-8.764147,-9.577295,-6.524882,0.081393,...,0.006116,0.054420,-0.273319,-2.128410,-5.646583,-8.764147,-9.577295,-6.524882,0.081393,96.754868
3496,-0.000062,0.005809,0.021386,-0.166383,-1.704553,-4.992775,-7.245802,-7.599645,-7.360784,-3.224979,...,0.005809,0.021386,-0.166383,-1.704553,-4.992775,-7.245802,-7.599645,-7.360784,-3.224979,85.600113
3497,0.000243,0.011048,0.041259,-0.389725,-2.908340,-7.789240,-11.448001,-11.546152,-7.852053,0.080314,...,0.011048,0.041259,-0.389725,-2.908340,-7.789240,-11.448001,-11.546152,-7.852053,0.080314,95.613312
3498,0.000082,0.006800,0.033747,-0.100591,-1.874741,-5.169100,-8.544689,-10.632514,-6.163387,-4.961085,...,0.006800,0.033747,-0.100591,-1.874741,-5.169100,-8.544689,-10.632514,-6.163387,-4.961085,85.756920


In [ ]:
df_combined_real.to_excel('/content/drive/MyDrive/Obeject_trajectory_research/df_results_arr_real.xlsx', index=False)

In [ ]:
df_imag = pd.read_excel('/content/drive/MyDrive/Obeject_trajectory_research/df_results_arr_imag.xlsx')
df_combined_imag = pd.concat([df_imag, df_results_arr_imag], axis=0, ignore_index=True)
df_combined_imag

,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,-0.000135,0.002926,0.085060,0.504006,1.317971,0.811208,-4.882224,-15.173388,-26.085423,-34.233383,...,-0.002926,-0.085060,-0.504006,-1.317971,-0.811208,4.882224,15.173388,26.085423,34.233383,0.0
1,-0.000143,0.002518,0.097440,0.671746,1.537988,-0.727061,-7.187041,-14.754610,-23.990673,-32.180557,...,-0.002518,-0.097440,-0.671746,-1.537988,0.727061,7.187041,14.754610,23.990673,32.180557,0.0
2,-0.000293,0.003367,0.146485,0.832502,1.313295,-1.345550,-8.487439,-21.498394,-31.210571,-38.292526,...,-0.003367,-0.146485,-0.832502,-1.313295,1.345550,8.487439,21.498394,31.210571,38.292526,0.0
3,-0.000227,0.003403,0.141411,0.866136,1.277426,-1.605051,-9.813153,-21.088516,-31.214546,-39.529587,...,-0.003403,-0.141411,-0.866136,-1.277426,1.605051,9.813153,21.088516,31.214546,39.529587,0.0
4,-0.000343,0.002417,0.117483,0.774329,1.050042,-1.069218,-7.448884,-16.913538,-26.984467,-33.908859,...,-0.002417,-0.117483,-0.774329,-1.050042,1.069218,7.448884,16.913538,26.984467,33.908859,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,-0.000064,-0.000472,0.073576,0.572311,1.300758,-0.010113,-5.668351,-13.993708,-23.827190,-27.187202,...,0.000472,-0.073576,-0.572311,-1.300758,0.010113,5.668351,13.993708,23.827190,27.187202,0.0
3496,-0.000047,0.003049,0.073170,0.502731,0.967626,-0.045670,-5.283481,-11.061962,-17.355501,-25.024450,...,-0.003049,-0.073170,-0.502731,-0.967626,0.045670,5.283481,11.061962,17.355501,25.024450,0.0
3497,-0.000269,-0.001586,0.139167,0.774784,1.493766,-1.240866,-9.132824,-18.428913,-28.494247,-38.685020,...,0.001586,-0.139167,-0.774784,-1.493766,1.240866,9.132824,18.428913,28.494247,38.685020,0.0
3498,-0.000369,0.005077,0.051388,0.524356,1.107400,-0.206374,-4.247838,-11.693213,-18.989727,-24.885193,...,-0.005077,-0.051388,-0.524356,-1.107400,0.206374,4.247838,11.693213,18.989727,24.885193,0.0


In [ ]:
df_combined_imag.to_excel('/content/drive/MyDrive/Obeject_trajectory_research/df_results_arr_imag.xlsx', index=False)